In [1]:
import numpy as np;
import pandas as pd;

## Initialize Parameters

In [2]:
## parameters and global variables

district = "ksg"

jC = 0.3
jM = 0.2
jP = 0.1

muC = 0.05
muM = 0.1
muP = 0.1

## Load and Initialize Data

In [9]:
## Load population data
initDataDF = pd.read_csv("../data/" + district + "_init_data.csv")

## Load distance data
distanceData = pd.read_csv("../data/" + district + "_distance_matrix.csv")

## Number of regions
r = len(initDataDF.index)

## Sort population data and distance data by name for easy matrix transformation
initDataDF.sort_values(by=['name'], inplace=True)
distanceData.sort_values(by=['fromName', 'toName'], inplace=True)

## Job column
initDataDF.loc[initDataDF['type']=='C','J'] = initDataDF[initDataDF['type']=='C']['S']*jC
initDataDF.loc[initDataDF['type']=='M','J'] = initDataDF[initDataDF['type']=='M']['S']*jM
initDataDF.loc[initDataDF['type']=='P','J'] = initDataDF[initDataDF['type']=='P']['S']*jP

initDataDF.loc[initDataDF['type']=='C','T'] = initDataDF[initDataDF['type']=='C']['S']*muC
initDataDF.loc[initDataDF['type']=='M','T'] = initDataDF[initDataDF['type']=='M']['S']*muM
initDataDF.loc[initDataDF['type']=='P','T'] = initDataDF[initDataDF['type']=='P']['S']*muP

initDataDF.head(30)

,name,type,S,E,I,H,R,J,T
0,Ajanur__Kasaragod,P,49153,0,0,0,0,4915.3,4915.3
1,Badiyaduka__Kasaragod,P,34207,0,0,0,0,3420.7,3420.7
2,Balal__Kasaragod,P,23670,0,0,0,0,2367.0,2367.0
3,Bedadka__Kasaragod,P,27868,0,0,0,0,2786.8,2786.8
4,Beloor__Kasaragod,P,10241,0,0,0,0,1024.1,1024.1
5,Chemmanad__Kasaragod,P,54747,0,0,0,0,5474.7,5474.7
6,Chengala__Kasaragod,P,56781,0,0,0,0,5678.1,5678.1
7,Cheruvathur__Kasaragod,P,27435,0,0,0,0,2743.5,2743.5
8,Delampadi__Kasaragod,P,22773,0,0,0,0,2277.3,2277.3
9,East Eleri__Kasaragod,P,25075,0,0,0,0,2507.5,2507.5


## Prepare Distance matrix

In [4]:
distanceMatrix = distanceData['value'].to_numpy().reshape(r,r)
#distanceMatrix[0][4]

## Function Definitions

In [5]:
def getSij(i, j, distanceMatrix, initDataDF):
    #print(distanceMatrix[i] <= distanceMatrix[i][j])
    return(initDataDF[distanceMatrix[i] <= distanceMatrix[i][j]]['J'].sum() - initDataDF.iloc[i]['J'] - initDataDF.iloc[j]['J'])
    
#getSij(63,23,distanceMatrix, initDataDF)

In [6]:
def getTij(i,j, distanceMatrix, initDataDF):
    Ni = initDataDF.iloc[i]['S']
    Ti = initDataDF.iloc[i]['T']

    if(i==j):
        return Ni - Ti;
    
    Ji = initDataDF.iloc[i]['J']
    Jj = initDataDF.iloc[j]['J']

    Sij = getSij(i,j,distanceMatrix, initDataDF)
    Tij = Ti * ((Ji * Jj) /((Ji + Sij)*(Ji + Jj + Sij)))
    return Tij
    
    

## Compute TMatrix

In [7]:
data = []

for i in range(0,r):
    for j in range(0,r):
        Tij = getTij(i,j,distanceMatrix, initDataDF)
        #print("(", i, ",", j, ") ",  initDataDF.iloc[i]['name'], " ", initDataDF.iloc[j]['name'], " " ,Tij)
        data.append({'iName': initDataDF.iloc[i]['name'], 'jName': initDataDF.iloc[j]['name'], 'Tij': Tij});
        
resultsDF = pd.DataFrame(data);


## Save output

In [8]:
resultsDF.to_csv("../output/" + district + "_tmatrix_results.csv")